In [1]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 6.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=d8320b6d129680cbcd3f21446b6ff54697ed73432ff78621eb4db55eef489c99
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [2]:
import tensorflow as tf
import os
print(tf.__version__)

from google.colab import drive
drive.mount('/content/drive')

2.15.0
Mounted at /content/drive


In [3]:
!mkdir video_test
%cd video_test
!unrar x '/content/drive/MyDrive/video test.rar'
%cd /content/

/content/video_test

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/video test.rar

Creating    video test                                                OK
Extracting  video test/armin_print.mp4                                     1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15%  OK 
Extracting  video test/armin_real.mp4                                     16% 17% 18% 19% 20% 21% 22% 23% 24% 25%  OK 
Extracting  video test/armin_screen.mp4                                   26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38%  OK 
Extracting  video test/china_print_1.avi                                  39%  OK 
Extracting  video test/china_print_2.avi                                  41%  OK 
Extracting  video test/china_real_1.

In [4]:
import cv2
import os

def extract_middle_frame(video_path, output_path):
    # Capture the video from the file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error opening video file {video_path}")
        return

    # Get the total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the middle frame index
    middle_frame_index = total_frames // 2

    # Set the current frame position to the middle frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame_index)

    # Read the middle frame
    ret, frame = cap.read()

    if ret:
        # Create the output file path
        video_filename = os.path.basename(video_path)
        video_name, _ = os.path.splitext(video_filename)
        output_file = os.path.join(output_path, f"{video_name}_middle_frame.jpg")

        # Save the frame as an image
        cv2.imwrite(output_file, frame)
        print(f"Saved middle frame of {video_path} to {output_file}")
    else:
        print(f"Failed to read the frame from {video_path}")

    # Release the video capture object
    cap.release()

def process_videos_in_directory(input_directory, output_directory):
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Process each video file in the directory
    for filename in os.listdir(input_directory):
        if filename.endswith((".mp4", ".avi", ".mkv", ".mov")):  # Add other video formats as needed
            # Check if the filename starts with "armin" or "farzam"
            if filename.lower().startswith(("armin", "farzam")):
                video_path = os.path.join(input_directory, filename)
                extract_middle_frame(video_path, output_directory)

# Specify the input directory containing videos and the output directory for images
input_directory = "/content/video_test/video test"
output_directory = "/content/faces"

# Process the videos
process_videos_in_directory(input_directory, output_directory)


Saved middle frame of /content/video_test/video test/farzam_screen.mp4 to /content/faces/farzam_screen_middle_frame.jpg
Saved middle frame of /content/video_test/video test/farzam_real.mp4 to /content/faces/farzam_real_middle_frame.jpg
Saved middle frame of /content/video_test/video test/armin_print.mp4 to /content/faces/armin_print_middle_frame.jpg
Saved middle frame of /content/video_test/video test/farzam_print.mp4 to /content/faces/farzam_print_middle_frame.jpg
Saved middle frame of /content/video_test/video test/armin_screen.mp4 to /content/faces/armin_screen_middle_frame.jpg
Saved middle frame of /content/video_test/video test/armin_real.mp4 to /content/faces/armin_real_middle_frame.jpg


In [6]:
import os
import cv2
from deepface import DeepFace

def extract_and_crop_faces(img_path, output_dir):
    # Load image
    img = cv2.imread(img_path)

    # Extract faces
    faces = DeepFace.extract_faces(img_path=img_path, detector_backend='ssd',enforce_detection=False)

    # Crop and save faces
    if faces:
        for face_idx, face in enumerate(faces):
            x, y, width, height = face["facial_area"]["x"], face["facial_area"]["y"], face["facial_area"]["w"], face["facial_area"]["h"]
            cropped_face = img[y:y+height, x:x+width]
            face_file = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(img_path))[0]}_cropped_face{face_idx + 1}.jpg")
            cv2.imwrite(face_file, cropped_face)
        print(f"Saved {len(faces)} faces from {img_path} to {output_dir}")
    else:
        print(f"No faces detected in image {img_path}.")

# Specify the source directory containing the images and the destination directory for cropped faces
source_dir = "/content/sample_data/ID_Cards"
output_dir = "/content/cropped_ID_Cards"

#source_dir = "/content/faces"
#output_dir = "/content/cropped_faces"

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process the images in the source directory
for filename in os.listdir(source_dir):
    if filename.endswith((".jpg", ".jpeg", ".png")):  # Add other image formats as needed
        img_path = os.path.join(source_dir, filename)
        extract_and_crop_faces(img_path, output_dir)


Saved 1 faces from /content/sample_data/ID_Cards/farzam_meli.jpg to /content/cropped_ID_Cards
Saved 1 faces from /content/sample_data/ID_Cards/armin_meli.jpg to /content/cropped_ID_Cards


In [9]:
import os
import csv
from deepface import DeepFace

# Paths to the folders containing the images
folder1 = "/content/cropped_faces"
folder2 = "/content/cropped_ID_Cards"

# Models, backend, and metric settings
models = ["Facenet512", "ArcFace"]
backend = "mtcnn"
metric = "cosine"

# Function to convert cosine distance to similarity
def cosine_similarity(distance: float) -> float:
    return 1 - distance

# Output CSV file
output_csv = "/content/similarity_report.csv"

# Open the CSV file for writing
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["face_img_names", "face_id_card_cropped_image_names", "similarity_facenet512", "similarity_arcface"])

    # Iterate over the images in the first folder
    for filename1 in os.listdir(folder1):
        if filename1.endswith(".jpg"):
            img1_path = os.path.join(folder1, filename1)

            # Iterate over the images in the second folder
            for filename2 in os.listdir(folder2):
                if filename2.endswith(".jpg"):
                    img2_path = os.path.join(folder2, filename2)

                    if os.path.exists(img2_path):
                        # Initialize similarities for both models
                        similarities = {}

                        for model in models:
                            # Verify the images
                            result = DeepFace.verify(
                                img1_path=img1_path,
                                img2_path=img2_path,
                                detector_backend=backend,
                                model_name=model,
                                distance_metric=metric,
                                enforce_detection=False
                            )
                            # Calculate similarity
                            distance = result['distance']
                            similarity = cosine_similarity(distance)
                            similarities[model] = similarity

                        # Write the result to the CSV file
                        writer.writerow([filename1, filename2, similarities["Facenet512"], similarities["ArcFace"]])

print(f"Similarity report saved to {output_csv}")


1/1 [==============================] - 0s 40ms/step
Similarity report saved to /content/similarity_report.csv
